In [1]:
%%capture
!sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!sudo apt-get -y update
!sudo apt-get -y install postgresql-14
!sudo service postgresql start
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import Error
from datetime import datetime

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
post = 5432
name = 'db'

point = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, post, name)

con = create_engine(point)

In [4]:
df_sql = pd.read_csv('/content/data_two_columns.csv')
df_sql.to_sql("sales", con, if_exists='replace', index=False, method='multi')

In [5]:
# def benchmark(func):        
#     def wrapper(*args, **kwargs):
#         start_time = datetime.now()
#         result = func(*args, **kwargs)
#         end_time = datetime.now()
#         print('Duration: {}'.format(end_time - start_time))
#         return result
#     return wrapper

In [6]:
# benchmark
def select_postgresql(sql):
    return pd.read_sql(sql, con)

In [7]:
sql = """select version();"""

In [8]:
%%time
select_postgresql(sql).head()

CPU times: user 3.02 ms, sys: 1.95 ms, total: 4.98 ms
Wall time: 5.79 ms


,version
0,PostgreSQL 14.2 (Ubuntu 14.2-1.pgdg18.04+1) on...


In [9]:
sql = """select s.* from sales as s limit 10"""

In [10]:
%%time
select_postgresql(sql).head()

CPU times: user 4.85 ms, sys: 0 ns, total: 4.85 ms
Wall time: 8.52 ms


,invoiceno,stockcode
0,536365,85123A
1,536365,71053
2,536365,84406B
3,536365,84029G
4,536365,84029E


In [11]:
def create_indexes() -> None:
    """Создание индексов """
    try:
        conn = psycopg2.connect(point)
        cursor = conn.cursor()
        cursor.execute(
            """create index index_invoiceno on sales(invoiceno);
               create index index_stockcode on sales(stockcode);"""
        )
        conn.commit()
    except (Exception, Error) as error:
        print("Ошибка при работе с PostgreSQL", error)
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с PostgreSQL закрыто")

In [12]:
create_indexes()

Соединение с PostgreSQL закрыто


### Вариант 1

In [13]:
sql = """with tbl_no_duplicates as (select s.invoiceno, 
                                           s.stockcode 
                                    from sales as s 
                                    group by s.invoiceno, 
                                             s.stockcode),
      
              tbl_list_code_combinations as (select t.invoiceno, (select array_agg(concat(cast(t1.* as text),', ',cast(t2.* as text)))
                                                                  from unnest(array_agg(t.stockcode)) as t1 
                                                                                     cross join unnest(array_agg(t.stockcode)) as t2
                                                                  where t1.* < t2.*) as agg
                                             from tbl_no_duplicates as t
                                            group by t.invoiceno)

select unnest(t.agg) as couple_stockcode, count(*)
from tbl_list_code_combinations as t
group by  unnest(t.agg) 
order by count(*) desc
limit 10"""

In [14]:
%%time
select_postgresql(sql).head()

CPU times: user 797 ms, sys: 84.8 ms, total: 882 ms
Wall time: 2min 32s


,couple_stockcode,count
0,"22386, 85099B",833
1,"22697, 22699",784
2,"21931, 85099B",733
3,"22411, 85099B",683
4,"20725, 22383",663


### Вариант 2

In [15]:
sql = """with tbl_no_duplicates as (select s.invoiceno, 
                                           s.stockcode 
                                    from sales as s 
                                    group by s.invoiceno, 
                                             s.stockcode),
      
              tbl_list_code_combinations as (select t1.stockcode as code1, t2.stockcode as code2
                                             from tbl_no_duplicates as t1, tbl_no_duplicates as t2
                                             where (t1.invoiceno = t2.invoiceno) and (t1.stockcode < t2.stockcode))

select t.code1, t.code2, count(*)
from tbl_list_code_combinations as t
 group by  t.code1, t.code2
 order by count(*) desc
 limit 10"""

In [16]:
%%time
select_postgresql(sql).head()

CPU times: user 204 ms, sys: 31.7 ms, total: 235 ms
Wall time: 38.8 s


,code1,code2,count
0,22386,85099B,833
1,22697,22699,784
2,21931,85099B,733
3,22411,85099B,683
4,20725,22383,663
